# 출처
- https://www.kaggle.com/agileteam/mock-exam1-type1-1-tutorial/notebook
- https://www.kaggle.com/agileteam/t2-3-adult-census-income-tutorial/notebook

# 작업형 1 모의고사

## 첫번째 데이터 부터 순서대로 50:50으로 데이터를 나누고, 앞에서 부터 50%의 데이터(이하, A그룹)는 'f1'컬럼의 결측치를 A그룹의 중앙값으로 채우고, 뒤에서부터 50% 데이터(이하, B그룹)는 'f1'컬럼의 결측치를 B그룹의 최대값으로 채운 후, A그룹과 B그룹의 표준편차 합을 구하시오
단, 소수점 첫째자리까지 구하시오 (둘째자리에서 반올림)

In [2]:
import pandas as pd
basic = pd.read_csv('basic1.csv')
# print(basic.head())
# print(basic.shape)

basic_A = basic.head(50)
basic_B = basic.tail(50)

A_median = basic_A.f1.median()
basic_A = basic_A.fillna(A_median)
basic_A_std = basic_A.f1.std()

B_max = basic_B.f1.max()
basic_B = basic_B.fillna(B_max)
basic_B_std = basic_B.f1.std()

result = round(basic_A_std + basic_B_std,1)
print(result)

42.0


## 'f4'컬럼을 기준 내림차순 정렬과 'f5'컬럼기준 오름차순 정렬을 순서대로 다중 조건 정렬하고나서 앞에서부터 10개의 데이터 중 'f5'컬럼의 최소값 찾고, 이 최소값으로 앞에서 부터 10개의 'f5'컬럼 데이터를 변경함. 그리고 'f5'컬럼의 평균값을 계산함¶
단 소수점 둘째자리까지 출력(셋째자리에서 반올림)

In [50]:
import warnings
warnings.filterwarnings('ignore')

basic_sort = basic.sort_values(['f4', 'f5'], ascending = [False, True]).reset_index()
basic_sort = basic_sort.drop('index', axis = 1)
basic_sort_top10 = basic_sort.head(10)

f5_min = basic_sort_top10.f5.min()
basic_sort_top10.f5 = f5_min
basic_sort_top10 = basic_sort_top10.reset_index()
basic_sort_top10 = basic_sort_top10.drop('index', axis = 1)

result = round(basic_sort.f5.mean(), 2)
print(result)

53.63


## 'age' 컬럼의 IQR방식을 이용한 이상치 수와 표준편차*1.5방식을 이용한 이상치 수 합을 구하시오¶
- IQR방식 : Q1 - 1.5 IQR, Q3 + 1.5 IQR에서 벗어나는 영역을 이상치라고 판단함 (Q1은 데이터의 25%, Q3는 데이터의 75% 지점임)
- 표준편차1.5방식: 평균으로부터 '표준편차1.5'를 벗어나는 영역을 이상치라고 판단함

In [70]:
# IQR
Q1 = basic.age.quantile(.25)
Median = basic.age.quantile(.5)
Q3 = basic.age.quantile(.75)
# print(Q1, Median, Q3)

IQR = Q3 - Q1
# print(IQR)
IQR_min_range = Q1 - IQR*1.5
IQR_max_range = Q3 + IQR*1.5
# print(IQR_min_range, IQR_max_range)

IQR_outlier_cond = (basic.age < IQR_min_range) | (basic.age > IQR_max_range)
IQR_outlier_cnt = basic[IQR_outlier_cond].shape[0]

# 표준편차*1.5 방식
Mean = basic.age.mean()
Std = basic.age.std()

std_min_range = Mean - Std*1.5
std_max_range = Mean + Std*1.5
# print(std_min_range, std_max_range)

std_outlier_cond = (basic.age < std_min_range) | (basic.age > std_max_range)
std_outlier_cnt = basic[std_outlier_cond].shape[0]

# 합치기!
answer = IQR_outlier_cnt + std_outlier_cnt
print(answer)   # 0 + 14

14


## 값 변경 및 2개 이상의 조건 Expected Questions
'f4'컬럼의 값이 'ESFJ'인 데이터를 'ISFJ'로 대체하고, 'city'가 '경기'이면서 'f4'가 'ISFJ'인 데이터 중 'age'컬럼의 최대값을 출력하시오!
- 데이터셋 : basic1.csv
- 오른쪽 상단 copy&edit 클릭 -> 예상문제 풀이 시작

In [13]:
basic.f4 = basic.f4.replace(['ESFJ'], ['ISFJ'])
cond = (basic.city == '경기') & (basic.f4 == 'ISFJ')
result = basic[cond].age.max()
print(result)

90.0


# # 작업형 2 모의고사: 성인 인구조사 소득 예측
- age: 나이
- workclass: 고용 형태
- fnlwgt: 사람의 대표성을 나타내는 가중치(final weight)
- education: 교육 수준
- education.num: 교육 수준 수치
- marital.status: 결혼 상태
- occupation: 업종
- relationship: 가족 관계
- race: 인종
- sex: 성별
- capital.gain: 양도 소득
- capital.loss: 양도 손실
- hours.per.week: 주당 근무 시간
- native.country: 국적
- income: 수익 (예측해야 하는 값)

In [1]:
# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("adult.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='income', null_name='?')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

C:\Users\Hansol\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


((26048, 15), (6513, 15), (26048, 2), (6513, 2))

In [126]:
# 1. 데이터 확인
# print(X_train.head())
# print(y_train.head())

# 2. 데이터 전처리
## 2.1. id 제거
X_train_no_id = X_train.drop('id', axis = 1)
X_test_no_id = X_test.drop('id', axis = 1)
y_train_no_id = y_train.drop('id', axis = 1)

### y_train_no_id의 속성값 변경 (<=50K은 0으로, >50K는 1로)
y_train_0_1 = y_train_no_id.replace(['<=50K', '>50K'], [0, 1])
# print(y_train_0_1)

## 2.2. 결측치 확인 및 채우기
# print(X_train_no_id.isnull().sum())
# print(X_test_no_id.isnull().sum())
# print(y_train_no_id.isnull().sum())    # 결측치 존재 x

### fillna(0)
X_train_fillna = X_train_no_id.fillna(0)
X_test_fillna = X_test_no_id.fillna(0)

## 2.3. One-hot encoding
X_train_onehot = pd.get_dummies(X_train_fillna)
X_test_onehot = pd.get_dummies(X_test_fillna)
# print(X_train_onehot.shape)
# print(X_test_onehot.shape)

### 이슈: 컬럼 개수가 안 맞음!
# print(set(X_train_onehot) - set(X_test_onehot))
### 결과: 'native.country_Holand-Netherlands', 'workclass_Never-worked' 열이 Test set에는 존재 X. 
###        따라서 이를 추가 해주자!
X_test_onehot['native.country_Holand-Netherlands'] = 0
X_test_onehot['workclass_Never-worked'] = 0

## 2.4. MinMaxScaler, StandardScaler
from sklearn.preprocessing import MinMaxScaler, StandardScaler
### MinMaxScaling
mm_scaler = MinMaxScaler()
mm_scaled_train = mm_scaler.fit_transform(X_train_onehot)
mm_scaled_train_df = pd.DataFrame(data = mm_scaled_train, columns = X_train_onehot.columns)

mm_scaled_test = mm_scaler.transform(X_test_onehot)
mm_scaled_test_df = pd.DataFrame(data = mm_scaled_test, columns = X_test_onehot.columns)

### StandardScaling
sc_scaler = StandardScaler()
sc_scaled_train = sc_scaler.fit_transform(X_train_onehot)
sc_scaled_train_df = pd.DataFrame(data = sc_scaled_train, columns = X_train_onehot.columns)

sc_scaled_test = sc_scaler.transform(X_test_onehot)
sc_scaled_test_df = pd.DataFrame(data = sc_scaled_test, columns = X_test_onehot.columns)

## 2.5. train_test_split (Training data set을 Training data와 Validation data로 나누기)
from sklearn.model_selection import train_test_split
y_train_no_id
mm_scaled_X_train, mm_scaled_X_test, mm_scaled_y_train, mm_scaled_y_test = train_test_split(mm_scaled_train_df, y_train_0_1, 
                                                                                            test_size=0.3, random_state = 123)

# 3. 모델 생성 및 평가
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier    # 너무 오래걸림
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, recall_score, mean_squared_error, classification_report, confusion_matrix

# # RF
# model = RandomForestClassifier()
# model.fit(mm_scaled_X_train, mm_scaled_y_train)
# score = model.score(mm_scaled_X_test, mm_scaled_y_test)
# print("RF's model_score :", score)
# y_pred = model.predict(mm_scaled_X_test)
# model_classification_report = classification_report(mm_scaled_y_test, y_pred)
# print("RF's classification_report :\n", model_classification_report)
# model_confusion_matrix = confusion_matrix(mm_scaled_y_test, y_pred)
# print("RF's classification_matrix :\n", model_confusion_matrix)
# print("\n")

# # KNN
# model = KNeighborsClassifier()
# model.fit(mm_scaled_X_train, mm_scaled_y_train)
# score = model.score(mm_scaled_X_test, mm_scaled_y_test)
# print("KNN's model_score :", score)
# y_pred = model.predict(mm_scaled_X_test)
# model_classification_report = classification_report(mm_scaled_y_test, y_pred)
# print("KNN's classification_report :\n", model_classification_report)
# model_confusion_matrix = confusion_matrix(mm_scaled_y_test, y_pred)
# print("KNN's classification_matrix :\n", model_confusion_matrix)
# print("\n")

# # XGB
# model = XGBClassifier()
# model.fit(mm_scaled_X_train, mm_scaled_y_train)
# score = model.score(mm_scaled_X_test, mm_scaled_y_test)
# print("XGB's model_score :", score)
# y_pred = model.predict(mm_scaled_X_test)
# model_classification_report = classification_report(mm_scaled_y_test, y_pred)
# print("XGB's classification_report :\n", model_classification_report)
# model_confusion_matrix = confusion_matrix(mm_scaled_y_test, y_pred)
# print("XGB's classification_matrix :\n", model_confusion_matrix)
# print("\n")

# 4. Final model 선정 후, predict
final_model = RandomForestClassifier()
final_model.fit(mm_scaled_X_train, mm_scaled_y_train)
y_test_pred = final_model.predict(mm_scaled_test)
print(mm_scaled_test)
y_test_pred
# y_test_pred_df = pd.DataFrame({'income_50K_이상': y_test_pred})

X_test[['id']].reset_index()

# X_test_id = X_test[['id']].reset_index()
# X_test_id = X_test_id.drop('index', axis = 1)

# submit = pd.concat([X_test_id, y_test_pred_df], axis = 1)
# submit.to_csv('submit(2021.12.04).csv')

[[0.56164384 0.06941634 0.53333333 ... 0.         0.         0.        ]
 [0.39726027 0.15943685 0.53333333 ... 0.         0.         0.        ]
 [0.68493151 0.06182339 0.86666667 ... 0.         0.         0.        ]
 ...
 [0.20547945 0.09562014 0.8        ... 0.         0.         0.        ]
 [0.42465753 0.10842083 0.93333333 ... 0.         0.         0.        ]
 [0.34246575 0.28584575 0.6        ... 0.         0.         0.        ]]


,index,id
0,20901,20901
1,14170,14170
2,1776,1776
3,30428,30428
4,8602,8602
...,...,...
6508,31222,31222
6509,10861,10861
6510,8929,8929
6511,2066,2066


In [76]:
# 시험환경에서는 아래와 같이 제공된다고 함
# import pandas as pd
# X_test = pd.read_csv("data/X_test.csv")
# X_train = pd.read_csv("data/X_train.csv")
# y_train = pd.read_csv("data/y_train.csv")